In [1]:
from pymongo import MongoClient
import folium
import pandas as pd
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import os

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

# Gaming Industry Research

#### First of all, I will investigate where gaming companies are localizated around the world. We need to take into consideration that decisions are based in the "companies" Dataset 

In [3]:
#I make a query in Mongo to get gaming companies offices

query = list(db.companies.find({"category_code":"games_video"},{"offices":1,"_id":0}))

In [4]:
gaming_industry = pd.DataFrame(query)

In [5]:
gaming_industry= gaming_industry.explode("offices")
gaming_industry

,offices
0,"{'description': None, 'address1': '8536 Nation..."
1,"{'description': 'Lala Headquarters', 'address1..."
2,"{'description': '', 'address1': '100 5th Ave F..."
3,"{'description': '', 'address1': '', 'address2'..."
4,"{'description': None, 'address1': None, 'addre..."
...,...
1079,"{'description': 'HQ', 'address1': '', 'address..."
1080,"{'description': 'New York', 'address1': '32-14..."
1080,"{'description': 'Houston', 'address1': '2208 C..."
1081,"{'description': '', 'address1': '163 Hastings ..."


In [6]:
# I get the country code from every single office 

def countryCode(row):
    try:
        offices = row.offices
        return {
            "countrycode":offices["country_code"],
        
        }
    except:
        return "N/A"

df = pd.DataFrame(gaming_industry)

df = pd.concat([df, df.apply(countryCode, axis=1, result_type="expand")], axis=1)
df = df.sort_values('countrycode')
df.head()

,offices,countrycode
754,"{'description': 'Mysoogal Inc', 'address1': 'n...",ARA
750,"{'description': 'Buenos Aires office', 'addres...",ARG
343,"{'description': '', 'address1': '999 Juana Man...",ARG
496,"{'description': 'Twigis.com', 'address1': '', ...",ARG
978,"{'description': 'Buenos Aires Office', 'addres...",ARG


In [7]:
# From "groupby" technique, I obtain the number of offices for each country 

cc_count=df.groupby("countrycode").count()

In [8]:
cc_count.head()

,offices
countrycode,
ARA,1
ARG,5
ARM,1
AUS,9
AUT,3


In [9]:
# I build two arrays to connect the country code with the number of offices in the countries

countrycode_values=np.asarray(list(sorted(set(df.countrycode.values))))

value_count=cc_count.offices.values


print(type(countrycode_values),type(value_count),len(countrycode_values),len(value_count))

country_code = pd.DataFrame({'country_code': countrycode_values, 'count': value_count})

country_code.head()


<class 'numpy.ndarray'> <class 'numpy.ndarray'> 54 54


,country_code,count
0,ARA,1
1,ARG,5
2,ARM,1
3,AUS,9
4,AUT,3


In [10]:
# MAP BUILDING WITH FOLIUM

In [11]:
country_geo ="INPUT/world-countries.json"

In [12]:
chart = folium.Map(location=[50,0], zoom_start=1.5)

In [13]:
hist_indicator="Video games companies around the world"

In [14]:
folium.Choropleth(geo_data=country_geo, data=country_code,
             columns=['country_code','count'],
             key_on='feature.id',
             fill_color='YlGnBu', fill_opacity=0.7, line_opacity=0.2,
             legend_name=hist_indicator).add_to(chart)

In [15]:
chart.save('plot_data.html')

In [16]:
from IPython.display import HTML
HTML('<iframe src=plot_data.html width=700 height=450></iframe>')

#### Since gaming is a software industry, it doesnt matter where in the world the companies are located at. Considering the Dataset, most of the video games companies are located in USA.  My conclusion is the following: If so  many companies from the crunch base firms dataset are based in United States, this means that there is not rather place to be located. I will reduce my research to that  area.



# US companies research

#### My next goal is to determinate whrere in US are located the companies founded after  2010 

In [19]:
query = list(db.companies.find({"founded_year":{"$gt":2010}},{"offices":1,"_id":0}))

In [20]:
US_companies = pd.DataFrame(query)

In [21]:
US_companies= US_companies.explode("offices")

In [22]:
def us_offices(row):
    try:
        offices = row.offices
        return {
            "countrycode":offices["country_code"],
            'state_code':offices["state_code"],
        
        
        }
    except:
        return "N/A"

df = pd.DataFrame(US_companies)

df = pd.concat([df, df.apply(us_offices, axis=1, result_type="expand")], axis=1)

In [23]:
    
df = pd.DataFrame(US_companies)

df = pd.concat([df, df.apply(us_offices, axis=1, result_type="expand")], axis=1)

In [24]:
df=df[df.countrycode == "USA"].groupby("state_code").offices.count()

In [25]:
df.head()

state_code
CA    24
CT     1
FL     2
IL     1
KY     1
Name: offices, dtype: int64

#### According to my research, tech companies from the Dataset, founded after 2010 in US, are basically located in the state of california. Because of that, I will limit the research to that area.

# California research 

#### I determine that the less flexible condition for the workers preferences is the airport location, so I decide that account managers shouldnt spend more than one hour to reach the airport. For a speed of 100 km/h in average, the company must be based within 100 km 

####   For the rest of preferences will be ponderate within 20 km as following :
#### Executives love starbucks -->10/123-->(20-20 * 0.08)--> Radius: 18 km
#### 30% of employees have a child-->26/123-->(20-20 * 0.2)-->Radius: 16 km
#### They all love going out to party-->87/123-->(20-20 * 0,70)--> Radius: 6 km 

In [26]:
import json
import requests
import os
from pymongo import MongoClient

In [27]:
client = MongoClient("mongodb://localhost/datamad0320")
db = client.get_database()

In [28]:
from dotenv import load_dotenv
load_dotenv()
apiKey = os.getenv("CLAVE_API")
print("WE HAVE APIKEY") if apiKey else print("NO APIKEY FOUND")

WE HAVE APIKEY


### AIRPORTS WITHIN 100 KM ✈️

In [29]:
url=f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=airports+in+california&key={apiKey}"

In [324]:
res = requests.get(url)

In [31]:
airports=res.json()

In [33]:
addresses = list(map(lambda e: e["formatted_address"],airports["results"]))

In [35]:
def geocode(address):

    res = requests.get(f"https://geocode.xyz/{address}",params={"json":1})
    data = res.json()
    print(res)
    try:
            return {
                "type":"Point",
                "coordinates": [float(data["longt"]), float(data["latt"])]
        }
    except:
        return None

In [114]:
coordinates = list(map(geocode,addresses))

In [115]:
coordinates=[e for e in coordinates if e!=None]

In [116]:
def getOfficeNear(point, maxDist=100000):
    return {
       "office": {
         "$near": {
           "$geometry": point,
           "$maxDistance": maxDist,
         }
       }
    }

In [117]:
query=[getOfficeNear(e) for e in coordinates]

In [118]:
for e in query:
    res = list(db.companies2.find(e, {"_id":0}))
    
len(res)

589

In [119]:
near_to_airports = pd.DataFrame(res)

In [120]:
# I keep the new database in a new collection 

near_to_airports.to_json("near_2_airports.json",orient="records")

### STARBUCKS WITHIN 18 KM ☕️

In [162]:
url=f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=starbucks+in+california&key={apiKey}"

In [325]:
res = requests.get(url)

In [164]:
data=res.json()

In [165]:
addresses = list(map(lambda e: e["formatted_address"],data["results"]))

In [134]:
coordinates = list(map(geocode,addresses))

In [130]:
coordinates=[e for e in coordinates if e!=None]

In [132]:
query=[getOfficeNear(e,maxDist=18000) for e in coordinates]

In [159]:
for e in query:
    res = list(db.companies3.find(e, {"_id":0}))
    
len(res)

94

In [153]:
near_to_starbucks = pd.DataFrame(res)

In [154]:
# I keep the new database in a new collection 

near_to_starbucks.to_json("near_to_starbucks.json",orient="records")

### CHILDCARE WITHIN 16 KM 👶🏻

In [167]:
url=f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=childcare+in+california&key={apiKey}"

In [326]:
#res = requests.get(url)

In [169]:
data=res.json()

In [170]:
addresses = list(map(lambda e: e["formatted_address"],data["results"]))

In [186]:
coordinates = list(map(geocode,addresses))

In [175]:
coordinates=[e for e in coordinates if e!=None]

In [181]:
query=[getOfficeNear(e,maxDist=16000) for e in coordinates]

In [193]:
for e in query:
    res = list(db.companies4.find(e, {"_id":0}))
    
len(res)

61

In [189]:
near_to_childcare = pd.DataFrame(res)

In [191]:
# I keep the new database in a new collection 

near_to_childcare.to_json("near_to_childcare.json",orient="records")

### DISCO WITHIN 6 KM 🎉

In [248]:
from dotenv import load_dotenv
load_dotenv()


True

In [261]:
#url = 'https://api.foursquare.com/v2/venues/explore'
#params = dict(
#client_id=os.getenv("CLIENT_ID"),
#client_secret=os.getenv("CLIENT_SECRET"),
#ll='37.38747,-122.05754',
#query='coffee',
#)
#resp = requests.get(url=url, params=params)
#data = json.loads(resp.text)

In [262]:
client_id=os.getenv("CLIENT_ID")
client_secret=os.getenv("CLIENT_SECRET")

In [312]:
url=f"https://api.foursquare.com/v2/venues/explore?client_id={client_id}&client_secret={client_secret}&v=20190425&near=california&query=party"


In [316]:
res=requests.get(url)
data = json.loads(resp.text)

In [319]:
addresses = list(map(lambda e: e["address"],data["results"]))

In [321]:
query=[getOfficeNear(e,maxDist=6000) for e in adressess]

In [322]:
for e in query:
    res = list(db.companies5.find(e, {"_id":0}))
    
len(res)

0

#### There is no place to go party within 6 km. However, the favourite vegan restaurant of the CEO (cafegratitude🥗) is just 500 meters far away from "SEO Intelligence", so the 4th floor of this building is the place finally selected
### COORDINATES: [-118.23341,34.041037]

In [323]:
#https://www.cafegratitude.com/